In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_regression

In [105]:
all_data = pd.read_csv('../data/all_data.csv')

In [106]:
all_data.columns

Index(['Unnamed: 0', 'date', 'su26230_days_before_coupon',
       'su26224_days_before_coupon', 'su26222_days_before_coupon',
       'su26221_days_before_coupon', 'su26218_days_before_coupon', 'ecb_rate',
       'aluminum', 'brent', 'cbr_key_rate', 'eur_rub', 'moex_index', 'nickel',
       'rtsi', 'usd_rub', 'su26218', 'su26221', 'su26222', 'su26224',
       'su26230', 'gazp', 'gmkn', 'lkoh', 'magn', 'mgnt', 'moex', 'rosn',
       'rual', 'sber', 'vtbr', 'datetime', 'year_1', 'year_3', 'year_5',
       'year_10', 'year_15', 'year_20'],
      dtype='object')

In [107]:
all_data

,Unnamed: 0,date,su26230_days_before_coupon,su26224_days_before_coupon,su26222_days_before_coupon,su26221_days_before_coupon,su26218_days_before_coupon,ecb_rate,aluminum,brent,...,rual,sber,vtbr,datetime,year_1,year_3,year_5,year_10,year_15,year_20
0,0,2020-01-10,89,145,103,89,82,"0,25",1806,"65,02",...,"31,7","258,19","0,0476",18:39:57,5.48,5.69,5.90,6.26,6.48,6.61
1,1,2020-01-13,86,142,100,86,79,"0,25",1798,"64,23",...,"32,2","262,4","0,04806",18:39:57,5.45,5.64,5.84,6.18,6.39,6.52
2,2,2020-01-14,85,141,99,85,78,"0,25",1809,"64,42",...,"32,83","259,05","0,04726",18:39:47,5.46,5.68,5.89,6.26,6.47,6.60
3,3,2020-01-15,84,140,98,84,77,"0,25",1801,64,...,"32,8","256,31","0,046805",18:39:46,5.44,5.71,5.93,6.30,6.50,6.63
4,4,2020-01-16,83,139,97,83,76,"0,25",1812,"64,62",...,"33,575","257,51","0,047825",18:39:19,5.46,5.70,5.91,6.29,6.50,6.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,884,2023-12-21,104,160,118,104,97,"4,75",2251,"79,39",...,"34,995","265,8","0,02303",18:39:58,12.23,11.62,11.77,11.93,12.00,12.03
885,885,2023-12-22,103,159,117,103,96,"4,75",2326,"79,07",...,"34,75","270,55","0,022935",18:39:57,12.15,11.91,11.88,11.90,11.91,11.92
886,886,2023-12-27,98,154,112,98,91,"4,75",2400,"79,54",...,"34,275","271,55","0,022925",18:39:57,12.13,12.03,12.01,12.03,12.05,12.05
887,887,2023-12-28,97,153,111,97,90,"4,75",2371,"77,15",...,"34,025",272,"0,02277",18:39:57,12.59,12.15,11.90,11.80,11.74,11.68


In [14]:
risk_factors_dtc ={
'sber': [], #сбер
'vtbr': [], #втб
'rual': [], #русал
'rosn': ['brent', 'eur_rub', 'usd_rub'], #роснефть
'moex': [], #мосбиржа
'mgnt': [], #магнит
'magn': [], #ММК
'lkoh': [], #лукойл
'gmkn': [], #гмк норникель
'gazp': [] #газпром

}

In [63]:
def calc_mutual_info(y, X):
    """
    Selects top N features by Mutual Information with target
    """
    # determine the mutual information
    mutual_info = mutual_info_regression(X.fillna(0), y)
    mutual_info = pd.Series(mutual_info)
    mutual_info.index = X.columns
    return list(mutual_info.sort_values(ascending=False).index)

In [78]:
def select_risk_factors(factor, risk_factors_list):
    """
    Функция принимает на вход колонку-название актива и список его возможных риск-факторов из словаря 
    Отсеиваются факторы с корреляцией между фактором и таргетом < 0.05
    Смотрится взаимная корреляция факторов, если она превышает 0.9, то оставляется один фактор с наибольшей MI с таргетом
    """
    target = all_data[factor].values
    risk_factors = all_data[risk_factors_list]
    drop_list_idx = []
    corr_list = []
    #считаем обычную корреляцию с таргетом
    for i in range(len(risk_factors.columns)):
        corr_with_target = np.corrcoef(x = risk_factors[risk_factors_list[i]].values, y = target)[0][1]
        corr_list.append(corr_with_target)
        if corr_with_target < 0.05: #дропаем, если меньше 0.05
            drop_list_idx.append(i)
    
    #ранжируем фичи по MI с таргетом
    top_mutual_information_list = calc_mutual_info(target, risk_factors)

    #смотрим взаимную корреляцию фичей
    mutual_correlation = np.corrcoef(x = risk_factors, rowvar=False)
    a = np.where(mutual_correlation >= 0.9)[0]
    b = np.where(mutual_correlation >= 0.9)[1]
    correlated_features = []
    for i in range(len(a)):
        if a[i]!= b[i]:
            correlated_features.append(tuple(sorted([a[i], b[i]])))
    correlated_features = set(correlated_features) #тут сет пар индексов скоррелированных фичей

    for pair in correlated_features: #смотрим, какая из фичей из пары на каком месте в ранжированном списке MI
        mi_1 =np.where(np.array(top_mutual_information_list) == risk_factors.columns[pair[0]]) 
        mi_2 =np.where(np.array(top_mutual_information_list) == risk_factors.columns[pair[1]])

        if mi_1 > mi_2: #если первый признак из пары менее связан с таргетом (дальше от начала списка MI)
            drop_list_idx.append(pair[0]) #дропаем первый признак
        else:
            drop_list_idx.append(pair[1])
    
    drop_list_names = list(risk_factors[risk_factors.columns[drop_list_idx]].columns)
    print(drop_list_names)
    return all_data[[factor] + risk_factors_list].drop(drop_list_names, axis=1)
        

In [79]:
select_risk_factors('rosn', risk_factors_dtc['rosn'])

['usd_rub']


,rosn,brent,eur_rub
0,473.50,65.02,68.0555
1,481.15,64.23,68.0450
2,475.50,64.42,67.8162
3,474.50,64.00,68.4213
4,480.95,64.62,68.3747
...,...,...,...
884,577.95,79.39,99.0480
885,583.20,79.07,100.5113
886,591.45,79.54,101.1178
887,593.95,77.15,101.3451
